In [1]:
# %pip install astropy
# %pip install scipy
# %pip install matplotlib
# %pip install astroquery
# %pip install skyfield

In [2]:
import numpy as np
import astropy.io.fits as fits
import matplotlib.pyplot as plt
from datetime import datetime, timezone, timedelta

# NEW: Import telescope and sensor configuration
from instrument_config import TelescopeConfig, SensorConfig

# Example usage of the new API
telescope = TelescopeConfig(
    focal_length_mm=1000.0,
    aperture_mm=100.0,
    image_circle_mm=50.0
)

sensor = SensorConfig(
    pixel_size_um=4.8,
    sensor_width_px=4144,
    sensor_height_px=2822,
    full_well_capacity_e=30000,
    quantum_efficiency=0.8,
    read_noise_e=5.0
)

computed_pixscale_arcsec = sensor.compute_pixel_scale_arcsec(telescope)
print(f"Pixel scale from config: {computed_pixscale_arcsec:.3f} arcsec/pixel")
print(f"Aperture area from config: {telescope.aperture_area_cm2:.2f} cm^2")
print(f"F-ratio from config: f/{telescope.f_ratio:.2f}")


In [3]:
from synthetic_image import (
    get_star_catalog,
    build_synthetic_wcs,
    world_to_pixel,
    mag_to_flux,
    add_psf_to_image,
    make_gaussian_psf,
    make_moffat_psf,
    sky_brightness_to_electrons,
    add_satellite_substepping,
    add_star_trails_skyfield,
    add_star_trails_fixed_camera
)
from skyfield.api import load, wgs84, Star

In [4]:
# Basic parameters
image_width = 4144
image_height = 2822

# Note: You could optionally override this with the computed_pixscale_arcsec above.
# For demonstration, we'll keep the originally hard-coded value.
ra_center = 88.29533   # degrees
dec_center = -5.364887 # degrees
rotation_degrees = -52.0

max_mag = 14.5 #15.5
seeing_arcsec = 5.5
pixscale_arcsec = 0.495  # you could do: computed_pixscale_arcsec

# Fictional camera/telescope parameters
aperture_area      = 324.29  # cm^2
quantum_efficiency = 0.800
exposure_time      = 2.0     # seconds
sky_mag_per_arcsec2 = 18.5   # mag/arcsec^2
mag_zero_point     = 15.5

In [5]:
# (Optional) If we want a final WCS matching the RA/Dec center and rotation:
wcs_obj, wcs_header = build_synthetic_wcs(
    image_width,
    image_height,
    ra_center,
    dec_center,
    pixscale_arcsec=pixscale_arcsec,
    rotation_degrees=rotation_degrees
)

In [6]:
# Create an empty image
synthetic_image = np.zeros((image_height, image_width), dtype=float)

In [ ]:
# Increase search radius to 2.0 deg to find more stars
star_catalog = get_star_catalog(ra_center, dec_center, radius=2.0, max_mag=max_mag)
if star_catalog is None or len(star_catalog) == 0:
    print('No stars returned by catalog query. You may need to use a brighter max_mag or larger radius.')
else:
    print(f'Successfully retrieved {len(star_catalog)} stars.')
fwhm_pixels = seeing_arcsec / pixscale_arcsec

In [ ]:
# Create a Moffat PSF
psf_kernel = make_moffat_psf(
    size=int(fwhm_pixels * 6),
    fwhm=fwhm_pixels,
    moffat_alpha=4
)
plt.imshow(psf_kernel, cmap='gray')
plt.colorbar()
plt.title('PSF Kernel')
plt.show()

In [ ]:
# Add sky background
sky_bg = sky_brightness_to_electrons(
    sky_mag_per_arcsec2,
    plate_scale_arcsec_per_pix=pixscale_arcsec,
    exposure_time=exposure_time,
    quantum_efficiency=quantum_efficiency,
    aperture_area=aperture_area,
    mag_zero_point=mag_zero_point
)
synthetic_image += sky_bg
plt.imshow(synthetic_image, cmap='gray')
plt.title('After Sky Background')
plt.show()

In [ ]:
# Suppose we have a 600s exposure, but shift it to a different time so the star might be above horizon.
# For example, pick ~2025-01-10 09:00 UTC to see if that lifts the star above horizon.
start_time = datetime(2025, 1, 10, 9, 0, 0, tzinfo=timezone.utc)
end_time = start_time + timedelta(seconds=exposure_time)
num_sub_steps = 20

# 1) Compute alt0, az0 that correspond to the RA/Dec center at start_time.
ts = load.timescale()
t0 = ts.from_datetime(start_time)
planets = load('de421.bsp')
earth = planets['earth']
site = earth + wgs84.latlon(34.05, -118.25, elevation_m=100.0)  # LA area, example
center_star = Star(ra_hours=ra_center/15.0, dec_degrees=dec_center)
app_center = site.at(t0).observe(center_star).apparent()
alt_center, az_center, dist_center = app_center.altaz()
alt0_guess = alt_center.degrees
az0_guess  = az_center.degrees
print(f'Computed alt0={alt0_guess:.2f}°, az0={az0_guess:.2f}° at start time')

In [ ]:
# 2) Use alt0_guess, az0_guess in add_star_trails_fixed_camera
if star_catalog is not None and len(star_catalog) > 0:
    add_star_trails_fixed_camera(
        image=synthetic_image,
        star_catalog=star_catalog,
        lat_deg=34.05,
        lon_deg=-118.25,
        alt_m=100.0,
        start_time=start_time,
        end_time=end_time,
        num_sub_steps=num_sub_steps,
        aperture_area=aperture_area,
        quantum_efficiency=quantum_efficiency,
        mag_zero_point=mag_zero_point,
        psf_kernel=psf_kernel,
        extinction=0.5,
        camera_rotation_deg=-90.0,
        arcsec_per_pix=pixscale_arcsec,
        alt0=alt0_guess,
        az0=az0_guess
    )
    plt.imshow(synthetic_image, cmap='gray', vmin=np.percentile(synthetic_image, 0.5), vmax=np.percentile(synthetic_image, 99.5))
    plt.title('Star Trails (Fixed Camera, alt0/az0 from ra_center/dec_center)')
    plt.show()
else:
    print('Skipping star trails because no star_catalog data was returned.')

In [ ]:
# Optional: Add noise
read_noise_e = 5.0
noisy_image = np.random.poisson(synthetic_image) + np.random.normal(loc=0.0, scale=read_noise_e, size=synthetic_image.shape)
noisy_image = np.fliplr(noisy_image)  # optional flip
plt.imshow(noisy_image, cmap='gray', vmin=np.percentile(noisy_image, 0.5), vmax=np.percentile(noisy_image, 99.5))
plt.title('Noisy Image')
plt.show()

In [ ]:
# Save to FITS with WCS from earlier (just referencing ra_center/dec_center at start)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
wcs_header["EXPTIME"] = (exposure_time, "Exposure time in seconds")
wcs_header["DATE-OBS"] = (start_time.strftime("%Y-%m-%dT%H:%M:%S"), "UTC date/time of observation start")
wcs_header["TELESCOP"] = ("Simulator", "Telescope name")
wcs_header["PIXSCALE"] = (pixscale_arcsec, "Plate scale in arcsec/pixel")
wcs_header["RA"] = (ra_center, "Right Ascension in degrees")
wcs_header["DEC"] = (dec_center, "Declination in degrees")
wcs_header["RADECSYS"] = ("ICRS", "Right Ascension and Declination system")

hdu = fits.PrimaryHDU(data=noisy_image, header=wcs_header)
output_filename = f"output/synthetic_image_{timestamp}.fits"
hdu.writeto(output_filename, overwrite=True)
print(f"Synthetic image saved to {output_filename}")